# Import

In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from konlpy.tag import Okt
from PIL import Image

from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

import FinanceDataReader as fdr
from nltk.corpus import reuters
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# 네이버 크롤링용 함수 생성

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    


# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}


#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

# 네이버 뉴스 크롤링 진행

In [ ]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

In [ ]:
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
        news_date = news_date[0:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        news_date = news_date[0:10]
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|██████████| 70/70 [00:21<00:00,  3.31it/s]

검색된 기사 갯수: 총  150 개
news_title:  70
news_url:  70
news_contents:  70
news_dates:  70


In [ ]:
#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
news_df.to_csv('{}.csv'.format(search),encoding='utf-8-sig',index=False)

In [6]:
import os

path = './메타버스_뉴스/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

metaverse_news = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)
    metaverse_news = metaverse_news.append(data)

C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4088419958.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\cod

In [7]:
metaverse_news["date"]= metaverse_news["date"].str.replace(".", "-", )

C:\Users\coden\AppData\Local\Temp\ipykernel_8964\787848037.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metaverse_news["date"]= metaverse_news["date"].str.replace(".", "-", )


In [8]:
# date 기준 정렬해주기
metaverse_news = metaverse_news.sort_values('date')
metaverse_news= metaverse_news.reset_index()

In [9]:
# 불필요한 index열 제거
metaverse_news = metaverse_news.drop(['index'], axis = 1)

In [10]:
metaverse_news = metaverse_news.replace("None", np.nan, )
metaverse_news = metaverse_news.dropna(axis = 0)
metaverse_news.head()

,date,title,link,content
0,2022-10-11,"팅크웨어, 中 체리차에 증강현실 내비게이션 솔루션 공급",https://n.news.naver.com/mnews/article/029/000...,[\n\t\t\t팅크웨어는 중국 완성차 기업인 체리자동차가 새로 선보이는 아리조 8...
1,2022-10-11,증강 현실로 먼저 입어보기,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...
2,2022-10-11,'증강 현실 기능을 통해 내가 아바타로',https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...
3,2022-10-11,'증강 현실 기능을 통해 내가 아바타로',https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...
4,2022-10-11,증강 현실로 입어보는 옷,https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n[서울=뉴시스] 정병혁 기자 = 11일 서울 중구 동대문디자인플...


# 빅카인즈 뉴스 크롤링

In [11]:
import os

path = './메타버스_뉴스_빅카인즈/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')]

metaverse_news = pd.DataFrame()
for i in file_list_py:
    data = pd.read_excel(path + i)
    data = data.iloc[:,0:3]
    metaverse_news = metaverse_news.append(data)

#중복 행 지우기
metaverse_news = metaverse_news.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(metaverse_news))

C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)
C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)


중복 제거 후 행 개수:  31523


C:\Users\coden\AppData\Local\Temp\ipykernel_8964\4202904727.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metaverse_news = metaverse_news.append(data)


In [12]:
metaverse_news.head()

,일자,제목,본문
0,20221112,걸핏하면 퇴사한다던 김 과장...그 얘기 쏙들어간 이유 [더테크웨이브],테크업계 혹독한 ‘겨울의 시간’\n요즘 국내 테크 업계에선 혹독한 ‘겨울의 시간’이...
1,20221112,"이승현 “미스코리아 진 당선, ‘고대생’ 덕도 있을 것” [인터뷰]","수영복 심사 사자머리 다 사라져...변화 추구 \n정시로 고대 입학, 꾸준히 공부한..."
2,20221112,"나인아이, 'Young Boy' VFX 퍼포먼스 비디오 기습 공개! '다이내믹 퍼포먼스'",그룹 나인아이(NINE.i)가 신개념 퍼포먼스 비디오로 독창적인 행보를 이어가고 있...
3,20221112,치열해지는 반도체 경쟁 '최종병기 활'을 지켜라 [더 머니이스트-Dr.J’s Chi...,트럼프 정부 2년 간 중국과 무역 갈등을 빚었던 미국은 다시 무역 적자가 다시 늘어...
4,20221111,[사설] 韓 美 간 의제된 北 사이버 위협 범국가적 컨트롤타워 시급하다,제이크 설리번 미국 백악관 국가안보보좌관은 다음주 캄보디아 프놈펜에서의 아세안 주요...


In [13]:
metaverse_news.columns = ['date', 'title', 'content']

In [14]:
# date 기준 정렬해주기
metaverse_news = metaverse_news.sort_values('date')
metaverse_news = metaverse_news.reset_index()

In [15]:
# 불필요한 index열 제거
metaverse_news = metaverse_news.drop(['index'], axis = 1)

In [16]:
for i in range(len(metaverse_news.date)):
    metaverse_news.date.iloc[i] = str(metaverse_news.date.iloc[i])

C:\Users\coden\AppData\Local\Temp\ipykernel_8964\2331600958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metaverse_news.date.iloc[i] = str(metaverse_news.date.iloc[i])


In [17]:
metaverse_news.head()

,date,title,content
0,20211112,"코인원, 연대 전략마케팅 학회와 산학협력 프로젝트",국내 암호화폐 거래소 코인원이 지난 1일부터 11일까지 코인원의 산업 경쟁력 강화를...
1,20211112,[시시비비]디지털 경쟁력 키울 정책,"‘메타버스 채용 부스마다 구직자 북적’, ‘메타버스로 업무 보는 시대’ 최근에 많은..."
2,20211112,“퀵커머스로 맛 본 편리함 못끊는다” [P-코노미가 온다 ③Platform],“처음에는 신종 코로나바이러스 감염증(코로나19)라는 환경적인 변화가 ‘퀵커머스’ ...
3,20211112,"공공기관 교육에 메타버스? 근로복지공단, 가상 인재개발원 구축",[머니투데이 세종=오세중 기자] \n공공기관 대상 교육에도 메타버스 플랫폼이 도입된...
4,20211112,"일부터 놀이까지 ‘컴투버스’에서 코엑스아쿠아리움, 디지털로 옮겨온다",컴투스(078340)는 계열사 위지윅스튜디오 얼반웍스가 서울오션아쿠아리움과 공동사업...


# 메타버스 관련주 등락률 가져오기

In [112]:
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

In [28]:
from datetime import datetime
from pandas_datareader import data as pdr
import pandas as pd
import xlrd

start=datetime(2020,1,20)


samsung=pdr.DataReader('000660.KS','yahoo',start)

stock_code=['036570','289220','030520','047080','035420','190510',
            '171090','347860','094360','096630','054210']

krx=fdr.StockListing('KRX')



stock_name=krx[['Code','Name']]



stock_name1=stock_name[stock_name['Code'].isin(stock_code)]

stock_name1=stock_name1.reset_index()

stock_name1=stock_name1.drop(['index'],axis=1)


price=[]

for i in range(len(stock_name1)):
    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),'2021-11-11', '2022-11-12'))
    
    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))


''' for i in range(len(stock_name1)):
    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),metaverse_news.date.iloc[0],metaverse_news.date.iloc[-1]))
    
    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))
 '''

" for i in range(len(stock_name1)):\n    price.append(fdr.DataReader('{}'.format(stock_name1['Code'][i]),metaverse_news.date.iloc[0],metaverse_news.date.iloc[-1]))\n    \n    price[i].to_csv('{}.csv'.format(stock_name1['Name'][i]))\n "

In [11]:
# 저장된 파일 /메타버스_종목 경로에 옮겨준 뒤 실행하기

In [44]:
import os

path = './메타버스_종목/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)[['Date', 'Change']]
    data.Change = data.Change*100
    data.columns = ['date', str(i[:-4]) + 'Change']
    df = pd.concat([df,data], axis = 1)
    df = df.replace(" ", np.nan)
    df = df.replace("-", "")
    df = df.replace(0, np.nan)
    df = df.fillna(0)

metaverse_stock = df.iloc[:,1::2]
metaverse_stock.insert(loc = 0, column = 'date', value = data['date'])

In [45]:
metaverse_stock.head()

,date,NAVERChange,나무가Change,선익시스템Change,알체라Change,에스코넥Change,엔씨소프트Change,이랜텍Change,자이언트스텝Change,칩스앤미디어Change,한글과컴퓨터Change,한빛소프트Change
0,2021-11-12,0.491400,7.231638,0.439560,-3.848746,1.994302,-9.033079,2.752294,0.502513,5.937500,3.787879,-1.737757
1,2021-11-15,0.488998,2.950474,8.096280,2.898703,3.351955,-7.692308,8.482143,21.500000,2.654867,3.163017,-2.572347
2,2021-11-16,-1.338200,2.354145,4.858300,4.421433,-2.162162,6.818182,-0.411523,0.754458,3.160920,3.066038,3.300330
3,2021-11-17,-1.109741,3.000000,9.073359,8.344479,0.552486,2.836879,5.785124,6.739278,0.000000,29.977117,7.827476
4,2021-11-18,1.496259,0.970874,3.716814,-1.896848,3.846154,4.827586,4.296875,1.977041,0.835655,3.345070,-4.444444


In [19]:
''' for i in range(1,len(metaverse_stock.columns)):
    for j in range(len(metaverse_stock)):
        if metaverse_stock.iloc[j,i] <= 0:
            metaverse_stock.iloc[j,i] = 0
        
        else:
            metaverse_stock.iloc[j,i] = 1

metaverse_stock.head() '''

,date,NAVERChange,나무가Change,선익시스템Change,알체라Change,에스코넥Change,엔씨소프트Change,이랜텍Change,자이언트스텝Change,칩스앤미디어Change,한글과컴퓨터Change,한빛소프트Change
0,2021-11-12,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1,2021-11-15,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
2,2021-11-16,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
3,2021-11-17,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,2021-11-18,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [46]:
# 뉴스 데이터와 맞게 date열 포맷
metaverse_stock["date"] = metaverse_stock["date"].str.replace("-", "", )

# 메타버스 관련주 일일 거래량 / 상장 주식수 만들기

In [47]:
import os

path = './메타버스_종목/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i)[['Date', 'Volume']]
    data.columns = ['date', str(i[:-4]) + 'Volume']
    df = pd.concat([df,data], axis = 1)
    df = df.replace(" ", np.nan)
    df = df.replace("-", "")
    df = df.replace(0, np.nan)
    df = df.fillna(0)

metaverse_volume = df.iloc[:,1::2]
metaverse_volume.insert(loc = 0, column = 'date', value = data['date'])

In [48]:
# 유통 주식 수 계산하기 (상장 주식수 - 특수관계인의 보유 주식 수) 전자공시시스템DART에서 활용
네이버_유통주식수 = 164049085 - 14670489
나무가_유통주식수 = 15196341 - 4946880
선익시스템_유통주식수 = 9250771 - 4506240 
알체라_유통주식수 = 17950944 - 5644546
에스코넥_유통주식수 = 66279963 - 12639469
엔씨소프트_유통주식수 = 21954022 - 2631366
이랜텍_유통주식수 = 25291210 - 9748724
자이언트스텝_유통주식수 = 21996222 - 9591110
칩스앤미디어_유통주식수 = 9640315 - 2861438
한글과컴퓨터_유통주식수 = 25203830 - 7735557
한빛소프트_유통주식수 = 24822362 - 7894197

metaverse_volume.iloc[:,1] = metaverse_volume.iloc[:,1]/나무가_유통주식수 * 100
metaverse_volume.iloc[:,2] = metaverse_volume.iloc[:,2]/선익시스템_유통주식수 * 100
metaverse_volume.iloc[:,3] = metaverse_volume.iloc[:,3]/알체라_유통주식수 * 100
metaverse_volume.iloc[:,4] = metaverse_volume.iloc[:,4]/에스코넥_유통주식수 * 100
metaverse_volume.iloc[:,5] = metaverse_volume.iloc[:,5]/엔씨소프트_유통주식수 * 100
metaverse_volume.iloc[:,6] = metaverse_volume.iloc[:,6]/이랜텍_유통주식수 * 100
metaverse_volume.iloc[:,7] = metaverse_volume.iloc[:,7]/자이언트스텝_유통주식수 * 100
metaverse_volume.iloc[:,8] = metaverse_volume.iloc[:,8]/칩스앤미디어_유통주식수 * 100
metaverse_volume.iloc[:,9] = metaverse_volume.iloc[:,9]/한글과컴퓨터_유통주식수 * 100
metaverse_volume.iloc[:,10] = metaverse_volume.iloc[:,10]/한빛소프트_유통주식수 * 100
metaverse_volume.iloc[:,11] = metaverse_volume.iloc[:,11]/네이버_유통주식수 * 100

In [49]:
# 뉴스 데이터와 맞게 date열 포맷
metaverse_volume["date"] = metaverse_volume["date"].str.replace("-", "", )

In [50]:
final = pd.merge(metaverse_stock, metaverse_volume, on = 'date', how = 'inner')

In [51]:
final.head()

,date,NAVERChange,나무가Change,선익시스템Change,알체라Change,에스코넥Change,엔씨소프트Change,이랜텍Change,자이언트스텝Change,칩스앤미디어Change,...,나무가Volume,선익시스템Volume,알체라Volume,에스코넥Volume,엔씨소프트Volume,이랜텍Volume,자이언트스텝Volume,칩스앤미디어Volume,한글과컴퓨터Volume,한빛소프트Volume
0,20211112,0.491400,7.231638,0.439560,-3.848746,1.994302,-9.033079,2.752294,0.502513,5.937500,...,13.018526,1.585151,1.321613,0.714213,9.128109,18.359471,19.734685,5.703912,1.655088,0.769513
1,20211115,0.488998,2.950474,8.096280,2.898703,3.351955,-7.692308,8.482143,21.500000,2.654867,...,8.374358,4.047277,0.908014,1.374992,7.999441,15.936890,46.874460,4.829831,2.578236,0.365639
2,20211116,-1.338200,2.354145,4.858300,4.421433,-2.162162,6.818182,-0.411523,0.754458,3.160920,...,9.493309,9.840540,1.895574,1.151834,5.673584,6.137599,18.506487,5.127714,3.484902,1.941944
3,20211117,-1.109741,3.000000,9.073359,8.344479,0.552486,2.836879,5.785124,6.739278,0.000000,...,10.500511,23.273609,10.398325,1.047967,3.829510,7.955059,39.238535,3.803765,53.052933,4.358126
4,20211118,1.496259,0.970874,3.716814,-1.896848,3.846154,4.827586,4.296875,1.977041,0.835655,...,6.147541,16.016262,3.561509,4.327718,4.421944,10.800370,17.483338,2.583736,61.419416,1.741295


# KOSPI 가져오기

In [60]:
df_kospi200 = fdr.DataReader('KS11', '2021-11-12', '2022-11-12')
df_kospi200 = df_kospi200.reset_index()
df_kospi200 = pd.DataFrame(df_kospi200[['Date', 'Close']])
df_kospi200.columns = ['date', 'kospi_Close']
df_kospi200['kospi_Change'] = df_kospi200['kospi_Close'].pct_change()*100

df_kospi200.to_csv("kospi200.csv", index=False, encoding="utf-8-sig")

In [62]:
kospi200 = pd.read_csv('kospi200.csv')

In [63]:
# 뉴스 데이터와 맞게 date열 포맷
kospi200["date"] = kospi200["date"].str.replace("-", "", )

In [64]:
final = pd.merge(final, kospi200, on = 'date', how = 'inner')

# VIX 변동률 지수 가져오기 (VIX가 오르면 공포심이 증가, VIX가 내리면 포심이 하락) VIX가 20아래면 안정 스트레스가 적어짐, 30이상이면 공포

In [70]:
df_VIX = fdr.DataReader('VIX', '2021-11-12', '2022-11-12')
df_VIX = df_VIX.reset_index()
df_VIX = pd.DataFrame(df_VIX[['Date', 'Close']])
df_VIX.columns = ['date', 'VIX_Close']

df_VIX.to_csv("VIX.csv", index=False, encoding="utf-8-sig")

In [71]:
VIX = pd.read_csv('VIX.csv')

In [72]:
# 뉴스 데이터와 맞게 date열 포맷
VIX["date"] = VIX["date"].str.replace("-", "", )

In [73]:
final = pd.merge(final, VIX, on = 'date', how = 'inner')

# 환율 데이터 가져오기

In [74]:
df_USD = fdr.DataReader('USD/KRW', '2021-11-12', '2022-11-12')
df_USD = df_USD.reset_index()
df_USD = pd.DataFrame(df_USD[['Date', 'Close']])
df_USD.columns = ['date', 'USD_Close']

df_USD.to_csv("USD.csv", index=False, encoding="utf-8-sig")

In [75]:
USD = pd.read_csv('USD.csv')

In [76]:
# 뉴스 데이터와 맞게 date열 포맷
USD["date"] = USD["date"].str.replace("-", "", )

In [77]:
final = pd.merge(final, USD, on = 'date', how = 'inner')

# 뉴스 감성분석 진행

# 0. 코스피의 등락 여부를 바탕으로 초기 감성사전 구축

In [38]:
print('학습데이터 전체 개수: {}'.format(len(metaverse_news)))

학습데이터 전체 개수: 31523


In [39]:
# title 
okt = Okt()
n_ = []
title_rename = []
for i in range(len(metaverse_news)):
  if(i % 5000 ==0):
    print(i,"단계 완료")
  title_rename.append(re.sub("[\(\[].*?[\)\]]", "",metaverse_news.iloc[i]['title']))
  n_.append(' '.join(okt.nouns(metaverse_news.iloc[i]['title'])))
metaverse_news['nouns_t'] = n_
metaverse_news['title']=title_rename
metaverse_news = metaverse_news[metaverse_news['nouns_t']!='']

#2차 불용어 제거
metaverse_news['title'] = metaverse_news['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
metaverse_news['title'].replace('', np.nan, inplace=True)
metaverse_news = metaverse_news.dropna(how='any')

0 단계 완료
5000 단계 완료
10000 단계 완료
15000 단계 완료
20000 단계 완료
25000 단계 완료
30000 단계 완료


C:\Users\coden\AppData\Local\Temp\ipykernel_8964\1793411106.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  metaverse_news['title'] = metaverse_news['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [40]:
# content
okt = Okt()
n_ = []
content_rename = []
for i in range(len(metaverse_news)):
  if(i % 5000 ==0):
    print(i,"단계 완료")
  content_rename.append(re.sub("[\(\[].*?[\)\]]", "",metaverse_news.iloc[i]['content']))
  n_.append(' '.join(okt.nouns(metaverse_news.iloc[i]['content'])))
metaverse_news['nouns_c'] = n_
metaverse_news['content']=content_rename
metaverse_news = metaverse_news[metaverse_news['nouns_c']!='']

#2차 불용어 제거
metaverse_news['content'] = metaverse_news['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
metaverse_news['content'].replace('', np.nan, inplace=True)
metaverse_news = metaverse_news.dropna(how='any')

0 단계 완료
5000 단계 완료
10000 단계 완료
15000 단계 완료
20000 단계 완료
25000 단계 완료
30000 단계 완료


C:\Users\coden\AppData\Local\Temp\ipykernel_8964\2244014389.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  metaverse_news['content'] = metaverse_news['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [41]:
metaverse_news.head()

,date,title,content,nouns_t,nouns_c
0,20211112,코인원 연대 전략마케팅 학회와 산학협력 프로젝트,국내 암호화폐 거래소 코인원이 지난 일부터 일까지 코인원의 산업 경쟁력 강화를 주제...,코인 연대 전략 마케팅 학회 산학 협력 프로젝트,국내 암호 화폐 거래소 코인 지난 코인 산업 경쟁력 강화 주제 연세대 전략 마케팅 ...
1,20211112,디지털 경쟁력 키울 정책,메타버스 채용 부스마다 구직자 북적 메타버스로 업무 보는 시대 최근에 많은 눈길을 ...,시시비비 디지털 경쟁력 정책,메타 버스 채용 부스 구직 북적 메타 버스 업무 시대 최근 눈길 끌 기사 제목 전 ...
2,20211112,퀵커머스로 맛 본 편리함 못끊는다,처음에는 신종 코로나바이러스 감염증라는 환경적인 변화가 퀵커머스 도입을 가속화 했지...,퀵 커머스 맛 코 노미,처음 신종 코로나바이러스 감염증 코로나 환경 변화 퀵 커머스 도입 가속 번 맛 소비...
3,20211112,공공기관 교육에 메타버스 근로복지공단 가상 인재개발원 구축,공공기관 대상 교육에도 메타버스 플랫폼이 도입된다 충북 진천군에 소재한 근로복지공...,공공기관 교육 메타 버스 근로 복지 공단 가상 인재 개발 구축,머니투데이 세종 중 기자 공공기관 대상 교육 메타 버스 플랫폼 도입 충북 진천군 소...
4,20211112,일부터 놀이까지 컴투버스에서 코엑스아쿠아리움 디지털로 옮겨온다,컴투스는 계열사 위지윅스튜디오 얼반웍스가 서울오션아쿠아리움과 공동사업양해각서를 체결...,일 놀이 컴투 버스 코엑스 아쿠아리움 디지털로,컴투스 계 열사 위지윅 스튜디오 얼반웍스 오션 아쿠아리움 공동 사업 양해각서 를 체...


In [207]:
# 거래를 하지 않는 날에 대한 전처리 후 how = 'outer'로 바꿀 예정
final = pd.merge(metaverse_news, final, how = 'inner')

In [208]:
for i in range(6, len(final.columns)):
    print(final.iloc[:,i].value_counts())

0.0    14050
1.0    12818
Name: 나무가Change, dtype: int64
0.0    14935
1.0    11933
Name: 선익시스템Change, dtype: int64
0.0    15157
1.0    11711
Name: 알체라Change, dtype: int64
0.0    15215
1.0    11653
Name: 에스코넥Change, dtype: int64
0.0    15247
1.0    11621
Name: 엔씨소프트Change, dtype: int64
0.0    14078
1.0    12790
Name: 이랜텍Change, dtype: int64
0.0    14835
1.0    12033
Name: 자이언트스텝Change, dtype: int64
1.0    13469
0.0    13399
Name: 칩스앤미디어Change, dtype: int64
0.0    14681
1.0    12187
Name: 한글과컴퓨터Change, dtype: int64
0.0    15118
1.0    11750
Name: 한빛소프트Change, dtype: int64
3.142292     223
5.272902     210
5.406636     208
3.457177     192
6.153397     192
            ... 
7.584096      56
6.365164      56
7.190651      55
9.267180      55
15.467047     51
Name: NAVERVolume, Length: 236, dtype: int64
22.564865    223
10.500511    210
4.905943     208
14.532564    192
28.829109    192
            ... 
4.579589      56
1.901284      56
3.890395      55
2.508404      55
3.863079      51
Name:

In [212]:
up = Counter(final.iloc[:]['칩스앤미디어Change'])[1]
down = Counter(final.iloc[:]['칩스앤미디어Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

print("전일 대비 상승 데이터:", up_ratio)
print("전일 대비 하락 데이터:", down_ratio)

전일 대비 상승 데이터: 0.5013026648801548
전일 대비 하락 데이터: 0.49869733511984515


# title 감성사전

In [173]:
# 나무가 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_나무가:' , len(vocab_t))

up = Counter(final.iloc[:]['나무가Change'])[1]
down = Counter(final.iloc[:]['나무가Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_나무가 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['나무가Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_나무가[noun] = vocab_t_나무가[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_나무가[noun] = vocab_t_나무가[noun] - up_ratio
# 선익시스템 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_선익시스템:' , len(vocab_t))

up = Counter(final.iloc[:]['선익시스템Change'])[1]
down = Counter(final.iloc[:]['선익시스템Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_선익시스템 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['선익시스템Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_선익시스템[noun] = vocab_t_선익시스템[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_선익시스템[noun] = vocab_t_선익시스템[noun] - up_ratio
# 알체라 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_알체라:' , len(vocab_t))

up = Counter(final.iloc[:]['알체라Change'])[1]
down = Counter(final.iloc[:]['알체라Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_알체라 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['알체라Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_알체라[noun] = vocab_t_알체라[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_알체라[noun] = vocab_t_알체라[noun] - up_ratio
# 에스코넥 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_에스코넥:' , len(vocab_t))

up = Counter(final.iloc[:]['에스코넥Change'])[1]
down = Counter(final.iloc[:]['에스코넥Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_에스코넥 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['에스코넥Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_에스코넥[noun] = vocab_t_에스코넥[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_에스코넥[noun] = vocab_t_에스코넥[noun] - up_ratio
# 엔씨소프트 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_엔씨소프트:' , len(vocab_t))

up = Counter(final.iloc[:]['엔씨소프트Change'])[1]
down = Counter(final.iloc[:]['엔씨소프트Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_엔씨소프트 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['엔씨소프트Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_엔씨소프트[noun] = vocab_t_엔씨소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_엔씨소프트[noun] = vocab_t_엔씨소프트[noun] - up_ratio
# 이랜텍 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_이랜텍:' , len(vocab_t))

up = Counter(final.iloc[:]['이랜텍Change'])[1]
down = Counter(final.iloc[:]['이랜텍Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_이랜텍 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['이랜텍Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_이랜텍[noun] = vocab_t_이랜텍[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_이랜텍[noun] = vocab_t_이랜텍[noun] - up_ratio
# 자이언트스텝 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_자이언트스텝:' , len(vocab_t))

up = Counter(final.iloc[:]['자이언트스텝Change'])[1]
down = Counter(final.iloc[:]['자이언트스텝Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_자이언트스텝 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['자이언트스텝Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_자이언트스텝[noun] = vocab_t_자이언트스텝[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_자이언트스텝[noun] = vocab_t_자이언트스텝[noun] - up_ratio
# 칩스앤미디어 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_칩스앤미디어:' , len(vocab_t))

up = Counter(final.iloc[:]['칩스앤미디어Change'])[1]
down = Counter(final.iloc[:]['칩스앤미디어Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_칩스앤미디어 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['칩스앤미디어Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_칩스앤미디어[noun] = vocab_t_칩스앤미디어[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_칩스앤미디어[noun] = vocab_t_칩스앤미디어[noun] - up_ratio 
# 한글과컴퓨터 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_한글과컴퓨터:' , len(vocab_t))

up = Counter(final.iloc[:]['한글과컴퓨터Change'])[1]
down = Counter(final.iloc[:]['한글과컴퓨터Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_한글과컴퓨터 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한글과컴퓨터Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한글과컴퓨터[noun] = vocab_t_한글과컴퓨터[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한글과컴퓨터[noun] = vocab_t_한글과컴퓨터[noun] - up_ratio
# 한빛소프트 =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_한빛소프트:' , len(vocab_t))

up = Counter(final.iloc[:]['한빛소프트Change'])[1]
down = Counter(final.iloc[:]['한빛소프트Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_한빛소프트 = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한빛소프트Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한빛소프트[noun] = vocab_t_한빛소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_한빛소프트[noun] = vocab_t_한빛소프트[noun] - up_ratio
# NAVER =====================================================================
vocab_t = {}
cnt = 0
for i in final['nouns_t']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_t or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_t[i[j]] = 0
print('title_NAVER:' , len(vocab_t))

up = Counter(final.iloc[:]['NAVERChange'])[1]
down = Counter(final.iloc[:]['NAVERChange'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_t_NAVER = vocab_t
for i,w in enumerate(final['nouns_t']):      
            w = w.split(' ')
      
            if (final.iloc[i]['NAVERChange']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_NAVER[noun] = vocab_t_NAVER[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_t_NAVER[noun] = vocab_t_NAVER[noun] - up_ratio

title_나무가: 13892
title_선익시스템: 13892
title_알체라: 13892
title_에스코넥: 13892
title_엔씨소프트: 13892
title_이랜텍: 13892
title_자이언트스텝: 13892
title_칩스앤미디어: 13892
title_한글과컴퓨터: 13892
title_한빛소프트: 13892
title_NAVER: 13892


# content 감성사전

In [87]:
# 나무가 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_나무가:' , len(vocab_c))

up = Counter(final.iloc[:]['나무가Change'])[1]
down = Counter(final.iloc[:]['나무가Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_나무가 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['나무가Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_나무가[noun] = vocab_c_나무가[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_나무가[noun] = vocab_c_나무가[noun] - up_ratio
# 선익시스템 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_선익시스템:' , len(vocab_c))

up = Counter(final.iloc[:]['선익시스템Change'])[1]
down = Counter(final.iloc[:]['선익시스템Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_선익시스템 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['선익시스템Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_선익시스템[noun] = vocab_c_선익시스템[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_선익시스템[noun] = vocab_c_선익시스템[noun] - up_ratio
# 알체라 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_알체라:' , len(vocab_c))

up = Counter(final.iloc[:]['알체라Change'])[1]
down = Counter(final.iloc[:]['알체라Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_알체라 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['알체라Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_알체라[noun] = vocab_c_알체라[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_알체라[noun] = vocab_c_알체라[noun] - up_ratio
# 에스코넥 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_에스코넥:' , len(vocab_c))

up = Counter(final.iloc[:]['에스코넥Change'])[1]
down = Counter(final.iloc[:]['에스코넥Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_에스코넥 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['에스코넥Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_에스코넥[noun] = vocab_c_에스코넥[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_에스코넥[noun] = vocab_c_에스코넥[noun] - up_ratio
# 엔씨소프트 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_엔씨소프트:' , len(vocab_c))

up = Counter(final.iloc[:]['엔씨소프트Change'])[1]
down = Counter(final.iloc[:]['엔씨소프트Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_엔씨소프트 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['엔씨소프트Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_엔씨소프트[noun] = vocab_c_엔씨소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_엔씨소프트[noun] = vocab_c_엔씨소프트[noun] - up_ratio
# 이랜텍 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_이랜텍:' , len(vocab_c))

up = Counter(final.iloc[:]['이랜텍Change'])[1]
down = Counter(final.iloc[:]['이랜텍Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_이랜텍 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['이랜텍Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_이랜텍[noun] = vocab_c_이랜텍[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_이랜텍[noun] = vocab_c_이랜텍[noun] - up_ratio
# 자이언트스텝 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_자이언트스텝:' , len(vocab_c))

up = Counter(final.iloc[:]['자이언트스텝Change'])[1]
down = Counter(final.iloc[:]['자이언트스텝Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_자이언트스텝 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['자이언트스텝Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_자이언트스텝[noun] = vocab_c_자이언트스텝[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_자이언트스텝[noun] = vocab_c_자이언트스텝[noun] - up_ratio
# 칩스앤미디어 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_칩스앤미디어:' , len(vocab_c))

up = Counter(final.iloc[:]['칩스앤미디어Change'])[1]
down = Counter(final.iloc[:]['칩스앤미디어Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_칩스앤미디어 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['칩스앤미디어Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_칩스앤미디어[noun] = vocab_c_칩스앤미디어[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_칩스앤미디어[noun] = vocab_c_칩스앤미디어[noun] - up_ratio 
# 한글과컴퓨터 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_한글과컴퓨터:' , len(vocab_c))

up = Counter(final.iloc[:]['한글과컴퓨터Change'])[1]
down = Counter(final.iloc[:]['한글과컴퓨터Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_한글과컴퓨터 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한글과컴퓨터Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한글과컴퓨터[noun] = vocab_c_한글과컴퓨터[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한글과컴퓨터[noun] = vocab_c_한글과컴퓨터[noun] - up_ratio
# 한빛소프트 =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_빛소프트:' , len(vocab_c))

up = Counter(final.iloc[:]['한빛소프트Change'])[1]
down = Counter(final.iloc[:]['한빛소프트Change'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_한빛소프트 = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['한빛소프트Change']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한빛소프트[noun] = vocab_c_한빛소프트[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_한빛소프트[noun] = vocab_c_한빛소프트[noun] - up_ratio
# NAVER =====================================================================
vocab_c = {}
cnt = 0
for i in final['nouns_c']:
    i = i.split(" ")
    for j in range(len(i)):
        if i[j] in vocab_c or len(i[j]) <= 1:
            cnt = cnt + 1
            pass
        
        else:
            vocab_c[i[j]] = 0
print('content_NAVER:' , len(vocab_c))

up = Counter(final.iloc[:]['NAVERChange'])[1]
down = Counter(final.iloc[:]['NAVERChange'])[0] 
up_ratio = up / (up+down)
down_ratio = down / (up+down)

vocab_c_NAVER = vocab_c
for i,w in enumerate(final['nouns_c']):      
            w = w.split(' ')
      
            if (final.iloc[i]['NAVERChange']==1):
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_NAVER[noun] = vocab_c_NAVER[noun] + down_ratio            
            else:
                  for j in range(len(w)):
                        noun = w[j]
                        if len(noun)<=1:
                              continue
                        vocab_c_NAVER[noun] = vocab_c_NAVER[noun] - up_ratio

content_나무가: 25275
content_선익시스템: 25275
content_알체라: 25275
content_에스코넥: 25275
content_엔씨소프트: 25275
content_이랜텍: 25275
content_자이언트스텝: 25275
content_칩스앤미디어: 25275
content_한글과컴퓨터: 25275
content_빛소프트: 25275
content_NAVER: 25275


# title 기반 감성스코어 연산 

In [215]:
# 나무가 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_나무가):
          continue
        else:
          sent_score = sent_score + vocab_t_나무가[j]
    total.append(sent_score/len(w))
    
final['title_score_나무가'] = total

# 선익시스템 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_선익시스템):
          continue
        else:
          sent_score = sent_score + vocab_t_선익시스템[j]
    total.append(sent_score/len(w))
    
final['title_score_선익시스템'] = total

# 알체라 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_알체라):
          continue
        else:
          sent_score = sent_score + vocab_t_알체라[j]
    total.append(sent_score/len(w))
    
final['title_score_알체라'] = total

# 에스코넥 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_에스코넥):
          continue
        else:
          sent_score = sent_score + vocab_t_에스코넥[j]
    total.append(sent_score/len(w))
    
final['title_score_에스코넥'] = total

# 엔씨소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_엔씨소프트):
          continue
        else:
          sent_score = sent_score + vocab_t_엔씨소프트[j]
    total.append(sent_score/len(w))
    
final['title_score_엔씨소프트'] = total

# 이랜텍 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_이랜텍):
          continue
        else:
          sent_score = sent_score + vocab_t_이랜텍[j]
    total.append(sent_score/len(w))
    
final['title_score_이랜텍'] = total

# 자이언트스텝 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_자이언트스텝):
          continue
        else:
          sent_score = sent_score + vocab_t_자이언트스텝[j]
    total.append(sent_score/len(w))
    
final['title_score_자이언트스텝'] = total

# 칩스앤미디어 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_칩스앤미디어):
          continue
        else:
          sent_score = sent_score + vocab_t_칩스앤미디어[j]
    total.append(sent_score/len(w))
    
final['title_score_칩스앤미디어'] = total

# 한글과컴퓨터 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_한글과컴퓨터):
          continue
        else:
          sent_score = sent_score + vocab_t_한글과컴퓨터[j]
    total.append(sent_score/len(w))
    
final['title_score_한글과컴퓨터'] = total

# 한빛소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_한빛소프트):
          continue
        else:
          sent_score = sent_score + vocab_t_한빛소프트[j]
    total.append(sent_score/len(w))
    
final['title_score_한빛소프트'] = total

# NAVER =====================================================================
total = []
for i,w  in enumerate(final['nouns_t']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_t_NAVER):
          continue
        else:
          sent_score = sent_score + vocab_t_NAVER[j]
    total.append(sent_score/len(w))
    
final['title_score_NAVER'] = total

# content 기반 감성스코어 연산

In [89]:
# 나무가 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_나무가):
          continue
        else:
          sent_score = sent_score + vocab_c_나무가[j]
    total.append(sent_score/len(w))
    
final['content_score_나무가'] = total

# 선익시스템 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_선익시스템):
          continue
        else:
          sent_score = sent_score + vocab_c_선익시스템[j]
    total.append(sent_score/len(w))
    
final['content_score_선익시스템'] = total

# 알체라 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_알체라):
          continue
        else:
          sent_score = sent_score + vocab_c_알체라[j]
    total.append(sent_score/len(w))
    
final['content_score_알체라'] = total

# 에스코넥 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_에스코넥):
          continue
        else:
          sent_score = sent_score + vocab_c_에스코넥[j]
    total.append(sent_score/len(w))
    
final['content_score_에스코넥'] = total

# 엔씨소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_엔씨소프트):
          continue
        else:
          sent_score = sent_score + vocab_c_엔씨소프트[j]
    total.append(sent_score/len(w))
    
final['content_score_엔씨소프트'] = total

# 이랜텍 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_이랜텍):
          continue
        else:
          sent_score = sent_score + vocab_c_이랜텍[j]
    total.append(sent_score/len(w))
    
final['content_score_이랜텍'] = total

# 자이언트스텝 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_자이언트스텝):
          continue
        else:
          sent_score = sent_score + vocab_c_자이언트스텝[j]
    total.append(sent_score/len(w))
    
final['content_score_자이언트스텝'] = total

# 칩스앤미디어 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_칩스앤미디어):
          continue
        else:
          sent_score = sent_score + vocab_c_칩스앤미디어[j]
    total.append(sent_score/len(w))
    
final['content_score_칩스앤미디어'] = total

# 한글과컴퓨터 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_한글과컴퓨터):
          continue
        else:
          sent_score = sent_score + vocab_c_한글과컴퓨터[j]
    total.append(sent_score/len(w))
    
final['content_score_한글과컴퓨터'] = total

# 한빛소프트 =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_한빛소프트):
          continue
        else:
          sent_score = sent_score + vocab_c_한빛소프트[j]
    total.append(sent_score/len(w))
    
final['content_score_한빛소프트'] = total

# NAVER =====================================================================
total = []
for i,w  in enumerate(final['nouns_c']):
    sent_score = 0
    w= w.split(' ')
    
    for j in w:
        if(len(j)<=1):
          continue
        elif(j not in vocab_c_NAVER):
          continue
        else:
          sent_score = sent_score + vocab_c_NAVER[j]
    total.append(sent_score/len(w))
    
final['content_score_NAVER'] = total

# 사전 결과 확인해보기

In [ ]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

# score 결과 저장 및 확인

In [95]:
final.to_csv("final_vocab.csv", index=False, encoding="utf-8-sig")

# score 스케일링

In [96]:
# 스코어는 0~100 사이의 점수로 산출할 것이므로 min-max스케일러 사용
from sklearn.preprocessing import MinMaxScaler

minMaxScaler = MinMaxScaler()
minMaxScaler.fit(final.iloc[:,16:len(final.columns)])
final.iloc[:,16:len(final.columns)] = minMaxScaler.transform(final.iloc[:,16:len(final.columns)]) * 100
final.iloc[:,16:len(final.columns)].head()

,NAVERVolume,나무가Volume,선익시스템Volume,알체라Volume,에스코넥Volume,엔씨소프트Volume,이랜텍Volume,자이언트스텝Volume,칩스앤미디어Volume,한글과컴퓨터Volume,...,content_score_선익시스템,content_score_알체라,content_score_에스코넥,content_score_엔씨소프트,content_score_이랜텍,content_score_자이언트스텝,content_score_칩스앤미디어,content_score_한글과컴퓨터,content_score_한빛소프트,content_score_NAVER
0,3.149042,5.38195,4.179625,12.445197,1.706396,100.0,49.53088,34.012767,7.646488,2.3085,...,18.908452,48.346670,37.444774,48.810395,34.216452,35.940634,16.325239,35.531048,44.183018,25.783334
1,3.149042,5.38195,4.179625,12.445197,1.706396,100.0,49.53088,34.012767,7.646488,2.3085,...,45.912868,62.220877,48.811072,38.243753,34.612378,26.078002,42.911550,42.040757,32.310873,35.499072
2,3.149042,5.38195,4.179625,12.445197,1.706396,100.0,49.53088,34.012767,7.646488,2.3085,...,31.988471,52.203809,35.201279,43.632570,39.202381,31.150751,24.919441,39.257457,31.638106,31.514744
3,3.149042,5.38195,4.179625,12.445197,1.706396,100.0,49.53088,34.012767,7.646488,2.3085,...,38.288268,40.470758,31.542836,27.376006,14.793974,18.624164,34.807908,37.923100,28.942051,24.375124
4,3.149042,5.38195,4.179625,12.445197,1.706396,100.0,49.53088,34.012767,7.646488,2.3085,...,50.201196,59.006663,48.305105,31.073799,36.526041,36.590349,50.741752,45.345175,38.309614,42.997447


# 일별 Score 평균 저장하기

In [97]:
feer = final.groupby(['date'], as_index=False).mean()

# 일별 / 종목별 공포-탐욕지수 산출

In [98]:
feer.head()

,date,NAVERChange,나무가Change,선익시스템Change,알체라Change,에스코넥Change,엔씨소프트Change,이랜텍Change,자이언트스텝Change,칩스앤미디어Change,...,content_score_선익시스템,content_score_알체라,content_score_에스코넥,content_score_엔씨소프트,content_score_이랜텍,content_score_자이언트스텝,content_score_칩스앤미디어,content_score_한글과컴퓨터,content_score_한빛소프트,content_score_NAVER
0,20211112,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,36.634490,54.759566,52.177776,46.372174,42.817287,39.449660,35.525078,52.773928,36.315240,39.421614
1,20211115,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,37.561096,56.848863,50.528026,46.644090,41.911932,37.572201,35.638251,48.520092,36.279334,37.722040
2,20211116,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,39.630700,58.268592,50.795653,47.137284,37.196956,37.099788,36.457729,49.652811,39.767742,35.191379
3,20211117,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,37.624088,58.769022,53.308306,50.476435,43.742300,41.077760,33.589773,49.798687,41.434608,36.128938
4,20211118,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,37.645463,54.694895,52.081238,48.725020,41.014525,39.505998,34.281670,47.880678,39.604808,37.906057


In [ ]:
for i in range(0, 11):
    for day in range(0, len(feer)):
        feer.iloc[:,12]

In [51]:
feer.iloc[:,12]

0       3.149042
1       1.898690
2       4.373759
3       5.146742
4       8.082884
         ...    
231    29.862102
232    36.420047
233    21.580497
234    11.918227
235    48.297603
Name: NAVERVolume, Length: 236, dtype: float64